Задание:

С помощью метода get_prepared_all_vacancies_from_hh
загрузить 10000 вакансии с сайта hh.ru.

Договоритесь чтобы вакансии не повторялись.

Сохранить результат в виде csv файла с тремя столбцами:
1. id вакансии
2. сырой json
3. подготовленное описание вакансии

Не забывайте про обработку ошибок

In [12]:
from urllib.request import urlopen
import json
import re
import string
import concurrent.futures
import urllib

import nltk
from langdetect import detect
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer 
import numpy as np
import pandas as pd
import os
from glob import glob

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

def download_json(url):
  response = []  
  try:
    response = urlopen(url)
  except (urllib.error.HTTPError, urllib.error.URLError) as error:
    print("Something bad happened")
    return []
  else:
    data_json = []
    if(response.status == 200):
        data_json = json.loads(response.read())
    return data_json
download_json('https://api.hh.ru/vacancies/43551857')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\keken\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\keken\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\keken\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'id': '43551857',
 'premium': False,
 'billing_type': {'id': 'standard', 'name': 'Стандарт'},
 'relations': [],
 'name': 'Рабочий в производственные цеха',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '68', 'name': 'Омск', 'url': 'https://api.hh.ru/areas/68'},
 'salary': {'from': 24000, 'to': 41000, 'currency': 'RUR', 'gross': False},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Омск',
  'street': 'улица Будеркина',
  'building': '2',
  'description': None,
  'lat': 54.9591513865,
  'lng': 73.437212985,
  'raw': 'Омск, улица Будеркина, 2',
  'metro': None,
  'metro_stations': []},
 'allow_messages': True,
 'site': {'id': 'hh', 'name': 'hh.ru'},
 'experience': {'id': 'noExperience', 'name': 'Нет опыта'},
 'schedule': {'id': 'shift', 'name': 'Сменный график'},
 'employment': {'id': 'full', 'name': 'Полная занятость'},
 'department': None,
 'contacts': None,
 'description': '<p><strong>Обязанности:</strong></p> <ul> <li>Работа в пр

In [28]:
os.chdir('E:\\univer\\3 курс 2 семестр\\ml_labs\\ML_OmSTU\\notebooks\\vacc')

dfs = [pd.read_csv(f, index_col=[0], parse_dates=[0])
        for f in os.listdir(os.getcwd()) if f.endswith('csv')]

finaldf = pd.concat(dfs, axis=0).sort_index()
print(finaldf)
finaldf.to_csv("vaccTotal.csv", index=False)

                                                          raw  \
id                                                              
34551956.0  {'id': '34551956', 'premium': False, 'billing_...   
34551957.0  {'id': '34551957', 'premium': False, 'billing_...   
34551958.0  {'id': '34551958', 'premium': False, 'billing_...   
34551959.0  {'id': '34551959', 'premium': False, 'billing_...   
34551960.0  {'id': '34551960', 'premium': False, 'billing_...   
...                                                       ...   
34564952.0  {'id': '34564952', 'premium': False, 'billing_...   
34564953.0  {'id': '34564953', 'premium': False, 'billing_...   
34564954.0  {'id': '34564954', 'premium': False, 'billing_...   
34564955.0  {'id': '34564955', 'premium': False, 'billing_...   
44551857.0  {'id': '44551857', 'premium': False, 'billing_...   

                                                     prepared  
id                                                             
34551956.0  ['услов', 'соб

In [18]:
'''
РЕАЛИЗОВАТЬ remove_garbage
Используйте регулярные выражения
IN: str '<p>ТРЕБОВАНИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...'
OUT: str 'ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...'
'''
def remove_garbage(raw_text):
  return re.sub(r"<[^>]*>", "", raw_text)
remove_garbage('<p>ТРЕБОВАНИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...')

'ТРЕБОВАНИЯ:  - обязательно высшее или незаконченное высшее экономическое образование;...'

In [4]:
'''
РЕАЛИЗОВАТЬ tokenize
Используйте word_tokenize из nltk.tokenize
IN: str 'ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...'
OUT: list ['требования', ':', '-', 'обязательно', 'высшее', 'или', 'незаконченное', 'высшее', 'экономическое', 'образование']
'''
def tokenize(raw_text):
  return word_tokenize(raw_text)
tokenize('ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...')

['ТРЕБОВАНИЯ',
 ':',
 '-',
 'обязательно',
 'высшее',
 'или',
 'незаконченное',
 'высшее',
 'экономическое',
 'образование',
 ';',
 '...']

In [5]:
'''
РЕАЛИЗОВАТЬ to_base_form
Можно использовать и лемматизацию и стемминг: pymystem3, ntlk, Spacy, gensim
IN: str 'Красивая мама красиво мыла раму'
OUT: str 'красивый мама красиво мыть рама'
'''
def to_base_form(raw_text):
  stemmer = SnowballStemmer("russian")
  tokens = tokenize(raw_text)
  result = [stemmer.stem(word) for word in tokens]
  return ' '.join(result)
to_base_form('Красивая мама красиво мыла раму')

'красив мам красив мыл рам'

In [6]:
'''
РЕАЛИЗОВАТЬ remove_stop_words
Используйте stopwords из nltk.corpus 
IN: list ['требования', ':', '-', 'обязательно', 'высший', 'или', 'незаконченный', 'высший', 'экономический', 'образование']
OUT: list ['требование', 'обязательно', 'высший', 'незаконченный', 'высший', 'экономический', 'образование']
'''
def remove_stop_words(raw_text):
  text = list(filter(lambda word: word not in stopwords.words('russian'), raw_text))
  text = list(filter(lambda word: word not in string.punctuation, text))
  return text
remove_stop_words(['требования', ':', '-', 'обязательно', 'высший', 'или', 'незаконченный', 'высший', 'экономический', 'образование'])

['требования',
 'обязательно',
 'высший',
 'незаконченный',
 'высший',
 'экономический',
 'образование']

In [7]:
'''
РЕАЛИЗОВАТЬ get_prepared_vacancy_description_from_hh
Используйте реализованные ранее методы
IN: int 72323
OUT1: str vacancy_json dump
OUT2: list[list] [['требование', 'обязательно', 'высший', 'незаконченный', 'высший', 'экономический', 'образование', ...], [...], ...]
'''
def get_prepared_vacancy_from_hh(vacancy_id):
  base_url = 'https://api.hh.ru/vacancies/'
  vacancy_json = download_json(base_url + str(vacancy_id))
  if(len(vacancy_json)!=0):
    vacancy_description = vacancy_json['description']
    vacancy_description_sentences = vacancy_description.split(".")
    prepared_vacancy_description = []
    for sentence in vacancy_description_sentences:
      sentence = remove_garbage(sentence)
      if(detect(sentence) == 'ru'):
        tokens = tokenize(to_base_form(sentence))
        prepared_vacancy_description += remove_stop_words(tokens)  
        return vacancy_id, vacancy_json, prepared_vacancy_description
      else:
        return []
  else:
    return []
get_prepared_vacancy_from_hh(43551857)

(43551857,
 {'id': '43551857',
  'premium': False,
  'billing_type': {'id': 'standard', 'name': 'Стандарт'},
  'relations': [],
  'name': 'Рабочий в производственные цеха',
  'insider_interview': None,
  'response_letter_required': False,
  'area': {'id': '68', 'name': 'Омск', 'url': 'https://api.hh.ru/areas/68'},
  'salary': {'from': 24000, 'to': 41000, 'currency': 'RUR', 'gross': False},
  'type': {'id': 'open', 'name': 'Открытая'},
  'address': {'city': 'Омск',
   'street': 'улица Будеркина',
   'building': '2',
   'description': None,
   'lat': 54.9591513865,
   'lng': 73.437212985,
   'raw': 'Омск, улица Будеркина, 2',
   'metro': None,
   'metro_stations': []},
  'allow_messages': True,
  'site': {'id': 'hh', 'name': 'hh.ru'},
  'experience': {'id': 'noExperience', 'name': 'Нет опыта'},
  'schedule': {'id': 'shift', 'name': 'Сменный график'},
  'employment': {'id': 'full', 'name': 'Полная занятость'},
  'department': None,
  'contacts': None,
  'description': '<p><strong>Обязанно

In [8]:
'''
РЕАЛИЗОВАТЬ get_prepared_all_vacancies_from_hh
Используйте get_prepared_vacancy_from_hh и ThreadPoolExecutor из concurrent.futures
IN: list [int]
OUT: list [vacancy_id, raw_description, prepared_description]
'''
def get_prepared_all_vacancies_from_hh(vacancy_ids):
  result = []
  with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for id in vacancy_ids:
      futures.append(executor.submit(get_prepared_vacancy_from_hh, vacancy_id=id))
    for future in concurrent.futures.as_completed(futures):
      result.append(future.result())
  return result

In [30]:
df = pd.DataFrame({
                'id': [],
                'raw': [],
                'prepared': []
            })
result = get_prepared_all_vacancies_from_hh(np.arange(34563956, 34564956, 1))

Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something bad happened
Something b

In [31]:
print(len(result))

1000


In [32]:
for val in result:
    if(len(val) !=0):
      df.loc[len(df)] = val
df.to_csv("vacc18.csv", index=False)

E:\Programming\anaconda\envs\ml_labs\lib\site-packages\pandas\core\dtypes\cast.py:881: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


In [33]:
df

,id,raw,prepared
0,34563957.0,"{'id': '34563957', 'premium': False, 'billing_...","[федеральн, компан, сет, ломбард, магазин, циф..."
1,34563956.0,"{'id': '34563956', 'premium': False, 'billing_...","[обязан, подготовк, пис, вопрос, входя, компет..."
2,34563958.0,"{'id': '34563958', 'premium': False, 'billing_...","[97, лет, групп, компан, «, росгосстр, », явля..."
3,34563959.0,"{'id': '34563959', 'premium': False, 'billing_...","[адрес, мест, работ, ул, западносибирск, лесоб..."
4,34563965.0,"{'id': '34563965', 'premium': False, 'billing_...","[обязан, регистрац, корреспонденц, направлен, ..."
...,...,...,...
947,34564950.0,"{'id': '34564950', 'premium': False, 'billing_...","[консультант, SAP, логистик, требован, оп, раб..."
948,34564951.0,"{'id': '34564951', 'premium': False, 'billing_...","[консультант, SAP, логистик, требован, оп, раб..."
949,34564953.0,"{'id': '34564953', 'premium': False, 'billing_...","[консультант, SAP, логистик, требован, оп, раб..."
950,34564954.0,"{'id': '34564954', 'premium': False, 'billing_...","[консультант, SAP, логистик, требован, оп, раб..."
